# Graphs for paper for combinations

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import seaborn as sns

plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["font.size"] = 7
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['axes.linewidth'] = 1.0
plt.rcParams['axes.grid'] = True

In [3]:
SOCKSHOP_DATASET_ID = "9n6mf"
TRAINTICKET_DATASET_ID = "m9dgg"

In [4]:
import sys
sys.path.append('../')

In [5]:
import neptune

In [6]:
## download from neptune
START_ID, END_ID = 558, 591
SKIP_IDS = []
RUN_IDS = [f"TSDRLOC-{i}" for i in range(START_ID, END_ID + 1) if i not in SKIP_IDS]

In [7]:
RUNS = []
for run_id in RUN_IDS:
    run = neptune.init_run(
        project="yuuk1/tsdr-localization",
        with_id=run_id,
        capture_stdout=False,
        capture_stderr=False,
        capture_hardware_metrics=False,
    )
    if run["sys/trashed"].fetch() == True:
        run.stop()
        continue
    struct = run.get_structure()
    RUNS.append(struct)
    run.stop()

https://app.neptune.ai/yuuk1/tsdr-localization/e/TSDRLOC-558
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/yuuk1/tsdr-localization/e/TSDRLOC-558/metadata
https://app.neptune.ai/yuuk1/tsdr-localization/e/TSDRLOC-559
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/yuuk1/tsdr-localization/e/TSDRLOC-559/metadata
https://app.neptune.ai/yuuk1/tsdr-localization/e/TSDRLOC-560
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/yuuk1/tsdr-localization/e/TSDRLOC-560/metadata
https://app.neptune.ai/yuuk1/tsdr-localization/e/TSDRLOC-561
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!

In [8]:
len(RUNS)

32

In [12]:
def convert_to_dataset_name(target_app: str, middleware: bool, manually_selected: bool) -> str:
    match target_app, middleware, manually_selected:
        case "sock-shop", _, True:
            return "SS-small"
        case "sock-shop", False, False:
            return "SS-medium"
        case "sock-shop", True, False:
            return "SS-large"
        case "train-ticket", _, True:
            return "SS-small"
        case "train-ticket", False, False:
            return "TT-medium"
        case "train-ticket", True, False:
            return "TT-large"
        case _:
            raise ValueError(f"Unknown combination: {target_app}, {middleware}, {manually_selected}")

In [13]:
items = []
for run in RUNS:
    target_app = run["dataset"]["target_app"].fetch()
    middleware = run["dataset"]["metric_types"]["middlewares"].fetch()
    manually_selected = run["dataset"]["use_manually_selected_metrics"].fetch()

    reduction: str
    tsdr_params = run["parameters"]["tsdr"]
    enable_uni: bool = tsdr_params["enable_unireducer"].fetch()
    enable_multi: bool = tsdr_params["enable_multireducer"].fetch()
    if uni_method := tsdr_params.get("step1_model_name", False) or tsdr_params.get("step1_method_name", False):
        uni_method = uni_method.fetch()
    if multi_method := tsdr_params.get("step2_clustering_method_name", False) or tsdr_params.get("step2_dbscan_algorithm", False):
        multi_method = multi_method.fetch()
    if multi_dist_type := tsdr_params.get("step2_dbscan_dist_type", False):
        multi_dist_type = multi_dist_type.fetch()
    
    match (enable_uni, enable_multi, uni_method, multi_method, multi_dist_type):
        case (False, False, _, _, _):
            reduction = "None"
        case (False, True, _, "changepoint-kde", _):
            reduction = "TSifter"
        case (True, False, "zscore_nsigma", _, _):
            reduction = "NSigma"
        case (True, False, "birch_model", _, _):
            reduction = "Birch"
        case (True, False, "two_samp_test", _, _):
            reduction = "K-S test"
        case (True, False, "fluxinfer", _, _):
            reduction = "FluxInfer-AD"
        case (False, True, _, "hdbscan", "sbd"):
            reduction = "HDBSCAN-SBD"
        case (False, True, _, "hdbscan", "pearsonr"):
            reduction = "HDBSCAN-PEARSON"
        case (_, _, _, _, _):
            assert False, f"Unknown reduction method: {tsdr_params}"

    # 1. e-Diagnosis
    # 2. RCD
    # 3. CausalRCA+PageRank
    # 4. PC+RW-2
    # 5. GES+RW-2
    # 6. LiNGAM+RW-2
    # 7. PC+PageRank
    # 8. GES+PageRank
    # 9. LiNGAM+PageRank
    # 10. PC+HT
    # 11. GES+HT
    # 12. LiNGAM+HT
    localization: str
    loc_params = run["parameters"]
    if pyrca_method := loc_params.get("method", ""):
        pyrca_method = pyrca_method.fetch()
    if walk_method := loc_params.get("walk_method", ""):
        walk_method = walk_method.fetch()

    if pyrca_method == "epsilon_diagnosis":
        localization = "e-Diagnosis"
    elif loc_params.get("use_rcd", False):
        localization = "RCD"
    elif loc_params.get("use_causalrca", False):
        localization = "CausalRCA"
    elif pyrca_method == "pc" and walk_method == "rw-2":
        localization = "PC+RW-2"
    elif pyrca_method in ["ges", "fges"] and walk_method == "rw-2":
        localization = "GES+RW-2"
    elif pyrca_method == "lingam" and walk_method == "rw-2":
        localization = "LiNGAM+RW-2"
    elif pyrca_method == "pc" and walk_method == "pagerank":
        localization = "PC+PageRank"
    elif pyrca_method in ["ges", "fges"] and walk_method == "pagerank":
        localization = "GES+PageRank"
    elif pyrca_method == "lingam" and walk_method == "pagerank":
        localization = "LiNGAM+PageRank"
    elif pyrca_method == "pc" and walk_method == "ht":
        localization = "PC+HT"
    elif pyrca_method in ["ges", "fges"] and walk_method == "ht":
        localization = "GES+HT"
    elif pyrca_method == "lingam" and walk_method == "ht":
        localization = "LiNGAM+HT"
    else:
        assert False, f"Unknown localization method: {loc_params}"

    ac2 = run["scores"]["metric"]["AC_2"].fetch()
    ac5 = run["scores"]["metric"]["AC_5"].fetch()
    avg5 = run["scores"]["metric"]["AVG_5"].fetch()
    elapsed_time = run["elapsed_time"]["mean"]["elapsed_time"].fetch()
    items.append({"dataset": convert_to_dataset_name(target_app, middleware, manually_selected),
                  "reduction_method": reduction, "localization_method": localization,
                  "ac@2": ac2, "ac@5": ac5, "avg@5": avg5, "elapsed_time": elapsed_time})

In [14]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    loc_tsdr_df = pd.DataFrame(items)
    display(loc_tsdr_df.set_index(["dataset", "reduction_method", "localization_method"]).sort_index())

ac@2      ac@5     avg@5  \
dataset   reduction_method localization_method                                 
SS-large  Birch            RCD                  0.240602  0.375940  0.275188   
                           e-Diagnosis          0.000000  0.000000  0.000000   
          FluxInfer-AD     RCD                  0.229630  0.362963  0.274074   
                           e-Diagnosis          0.000000  0.000000  0.000000   
          HDBSCAN-PEARSON  RCD                  0.259259  0.370370  0.287407   
                           e-Diagnosis          0.000000  0.000000  0.000000   
          HDBSCAN-SBD      RCD                  0.222222  0.377778  0.288889   
                           e-Diagnosis          0.000000  0.000000  0.000000   
          K-S test         RCD                  0.222222  0.385185  0.275556   
                           e-Diagnosis          0.000000  0.000000  0.000000   
          NSigma           RCD                  0.251908  0.374046  0.287023   
                           e-Diagnosis          0.000000  0.000000  0.000000   
          None             RCD                  0.259259  0.414815  0.305185   
                           e-Diagnosis          0.000000  0.000000  0.000000   
          TSifter          RCD                  0.229630  0.362963  0.272593   
                           e-Diagnosis          0.029630  0.081481  0.051852   
SS-medium Birch            RCD                  0.488889  0.622222  0.533333   
                           e-Diagnosis          0.000000  0.000000  0.000000   
          FluxInfer-AD     RCD                  0.503704  0.629630  0.534815   
                           e-Diagnosis          0.000000  0.014815  0.002963   
          HDBSCAN-PEARSON  RCD                  0.496296  0.577778  0.518519   
                           e-Diagnosis          0.044444  0.237037  0.103704   
          HDBSCAN-SBD      RCD                  0.555556  0.622222  0.564444   
                           e-Diagnosis          0.022222  0.177778  0.072593   
          K-S test         RCD                  0.496296  0.607407  0.520000   
                           e-Diagnosis          0.000000  0.014815  0.002963   
          NSigma           RCD                  0.526718  0.656489  0.557252   
                           e-Diagnosis          0.000000  0.000000  0.000000   
          None             RCD                  0.511111  0.614815  0.534815   
                           e-Diagnosis          0.000000  0.007407  0.001481   
          TSifter          RCD                  0.562963  0.644444  0.586667   
                           e-Diagnosis          0.133333  0.340741  0.214815   

                                                elapsed_time  
dataset   reduction_method localization_method                
SS-large  Birch            RCD                    542.001034  
                           e-Diagnosis             40.748816  
          FluxInfer-AD     RCD                    650.102633  
                           e-Diagnosis             46.629614  
          HDBSCAN-PEARSON  RCD                    109.891170  
                           e-Diagnosis              8.752305  
          HDBSCAN-SBD      RCD                    105.818111  
                           e-Diagnosis              8.201547  
          K-S test         RCD                    649.253940  
                           e-Diagnosis             46.086799  
          NSigma           RCD                    485.698916  
                           e-Diagnosis             35.362005  
          None             RCD                    724.015596  
                           e-Diagnosis             53.063932  
          TSifter          RCD                    282.353169  
                           e-Diagnosis             23.852554  
SS-medium Birch            RCD                     82.270278  
                           e-Diagnosis              5.955241  
          FluxInfer-AD     RCD                     84.600981  
        

## Load tsdr

In [15]:
TSDR_START_ID, TSDR_END_ID = 457, 472
TSDR_RUN_IDS = [f"TSDR-{id}" for id in range(TSDR_START_ID, TSDR_END_ID + 1)]

In [16]:
TSDR_RUNS = []
for run_id in TSDR_RUN_IDS:
    run = neptune.init_run(
        project="yuuk1/tsdr",
        with_id=run_id,
        capture_stdout=False,
        capture_stderr=False,
        capture_hardware_metrics=False,
    )
    if run["sys/trashed"].fetch() == True:
        run.stop()
        continue
    struct = run.get_structure()
    TSDR_RUNS.append(struct)
    run.stop()

https://app.neptune.ai/yuuk1/tsdr/e/TSDR-457
Unexpected error occurred in Neptune background thread: Killing Neptune ping thread. Your run's status will not be updated and the run will be shown as inactive.


Exception in thread NeptunePing:
Traceback (most recent call last):
  File "/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.11/site-packages/neptune/common/backends/utils.py", line 65, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.11/site-packages/neptune/internal/backends/hosted_neptune_backend.py", line 435, in ping
    self.leaderboard_client.api.ping(
  File "/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.11/site-packages/neptune/internal/backends/swagger_client_wrapper.py", line 99, in __call__
    self.handle_neptune_http_errors(e.response, exception=e)
  File "/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.11/site-packages/neptune/internal/backends/swagger_client_wrapper.py", line 92, in handle_neptune_http_errors
    raise exception
  File "/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/py

Shutting down background jobs, please wait a moment...
Unexpected error occurred in Neptune background thread: Killing Neptune asynchronous thread. All data is safe on disk and can be later synced manually using `neptune sync` command.


Exception in thread NeptuneAsyncOpProcessor:
Traceback (most recent call last):
  File "/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.11/site-packages/neptune/common/backends/utils.py", line 65, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.11/site-packages/neptune/internal/backends/hosted_neptune_backend.py", line 642, in _execute_operations
    result = self.leaderboard_client.api.executeOperations(**kwargs).response().result
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.11/site-packages/neptune/internal/backends/swagger_client_wrapper.py", line 99, in __call__
    self.handle_neptune_http_errors(e.response, exception=e)
  File "/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.11/site-packages/neptune/internal/backends/swagger_client

Done!
Explore the metadata in the Neptune app:
https://app.neptune.ai/yuuk1/tsdr/e/TSDR-457/metadata
https://app.neptune.ai/yuuk1/tsdr/e/TSDR-458
Shutting down background jobs, please wait a moment...
Unexpected error occurred in Neptune background thread: Killing Neptune ping thread. Your run's status will not be updated and the run will be shown as inactive.
Unexpected error occurred in Neptune background thread: Killing Neptune asynchronous thread. All data is safe on disk and can be later synced manually using `neptune sync` command.


...

Unexpected error occurred in Neptune background thread: Killing Neptune asynchronous thread. All data is safe on disk and can be later synced manually using `neptune sync` command.


Exception in thread NeptuneAsyncOpProcessor:
Traceback (most recent call last):
  File "/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.11/site-packages/neptune/common/backends/utils.py", line 65, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.11/site-packages/neptune/internal/backends/hosted_neptune_backend.py", line 642, in _execute_operations
    result = self.leaderboard_client.api.executeOperations(**kwargs).response().result
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.11/site-packages/neptune/internal/backends/swagger_client_wrapper.py", line 99, in __call__
    self.handle_neptune_http_errors(e.response, exception=e)
  File "/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.11/site-packages/neptune/internal/backends/swagger_client

Done!
Explore the metadata in the Neptune app:
https://app.neptune.ai/yuuk1/tsdr/e/TSDR-471/metadata
https://app.neptune.ai/yuuk1/tsdr/e/TSDR-472
Unexpected error occurred in Neptune background thread: Killing Neptune asynchronous thread. All data is safe on disk and can be later synced manually using `neptune sync` command.
Unexpected error occurred in Neptune background thread: Killing Neptune ping thread. Your run's status will not be updated and the run will be shown as inactive.
Shutting down background jobs, please wait a moment...


Exception in thread NeptuneAsyncOpProcessor:
Traceback (most recent call last):
  File "/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.11/site-packages/neptune/common/backends/utils.py", line 65, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.11/site-packages/neptune/internal/backends/hosted_neptune_backend.py", line 642, in _execute_operations
    result = self.leaderboard_client.api.executeOperations(**kwargs).response().result
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.11/site-packages/neptune/internal/backends/swagger_client_wrapper.py", line 99, in __call__
    self.handle_neptune_http_errors(e.response, exception=e)
  File "/home/ubuntu/src/github.com/ai4sre/meltria-analyzer/.venv/lib/python3.11/site-packages/neptune/internal/backends/swagger_client

Done!
Explore the metadata in the Neptune app:
https://app.neptune.ai/yuuk1/tsdr/e/TSDR-472/metadata


In [39]:
tsdr_items = []
for run in TSDR_RUNS:
    target_app = run["dataset"]["target_app"].fetch()
    middleware = run["dataset"]["metric_types"]["middlewares"].fetch()
    manually_selected = run["dataset"]["use_manually_selected_metrics"].fetch()

    reduction: str
    tsdr_params = run["parameters"]
    enable_uni: bool = tsdr_params["enable_unireducer"].fetch()
    enable_multi: bool = tsdr_params["enable_multireducer"].fetch()
    if uni_method := tsdr_params.get("step1_model_name", False) or tsdr_params.get("step1_method_name", False):
        uni_method = uni_method.fetch()
    if multi_method := tsdr_params.get("step2_clustering_method_name", False) or tsdr_params.get("step2_dbscan_algorithm", False):
        multi_method = multi_method.fetch()
    if multi_dist_type := tsdr_params.get("step2_dbscan_dist_type", False):
        multi_dist_type = multi_dist_type.fetch()
    
    match (enable_uni, enable_multi, uni_method, multi_method, multi_dist_type):
        case (False, False, _, _, _):
            reduction = "None"
        case (False, True, _, "changepoint-kde", _):
            reduction = "TSifter"
        case (True, False, "zscore_nsigma", _, _):
            reduction = "NSigma"
        case (True, False, "birch_model", _, _):
            reduction = "Birch"
        case (True, False, "two_samp_test", _, _):
            reduction = "K-S test"
        case (True, False, "fluxinfer", _, _):
            reduction = "FluxInfer-AD"
        case (False, True, _, "hdbscan", "sbd"):
            reduction = "HDBSCAN-SBD"
        case (False, True, _, "hdbscan", "pearsonr"):
            reduction = "HDBSCAN-PEARSON"
        case (_, _, _, _, _):
            assert False, f"Unknown reduction method: {tsdr_params}"

    frr = run["scores"]["reduction_rate_mean"].fetch()
    rfr = run["scores"]["cause_metrics"]["recall_mandatory_mean"].fetch()
    rfp = run["scores"]["cause_metrics"]["proportion_mandatory_mean"].fetch()
    num_series = tuple([int (s) for s in run["scores"]["num_series"]["total"].fetch().split("/")])
    num_found_cause_metrics = run["scores"]["cause_metrics"]["num_mandatory_found_mean"].fetch()
    elapsed_time = run["scores"]["elapsed_time_min"].fetch()
    tsdr_items.append({"dataset": convert_to_dataset_name(target_app, middleware, manually_selected),
                  "reduction_method": reduction,
                  "FRR": frr, "RFR": rfr, "RFP": rfp, "elapsed_time": elapsed_time, "num_metrics(reduced)": num_series[0], "num_metrics(filtered)": num_series[1], "num_metrics(raw)": num_series[2], "num_found_cause_metrics": num_found_cause_metrics})

In [40]:
reduction_methods = [
    "None", "NSigma", "Birch", "K-S test", "FluxInfer-AD", "HDBSCAN-SBD", "HDBSCAN-PEARSON", "TSifter"]
metrics = ["FRR", "RFR", "RFP", "elapsed_time", "num_metrics(reduced)"]

with pd.option_context("display.max_rows", None, "display.max_columns", None):
    multi_level_indexes = pd.MultiIndex.from_product(
        [["SS-medium", "SS-large"], reduction_methods], names=['dataset', 'reduction_method'])
    multi_level_cols = pd.MultiIndex.from_product(
        [["SS-medium", "SS-large"], metrics], names=['dataset', 'metric'])
    tsdr_df = pd.DataFrame(tsdr_items)
    df_pivot = tsdr_df.set_index(['reduction_method', 'dataset']).unstack('dataset')
    df_pivot = df_pivot.swaplevel(axis=1).sort_index(axis=1).reindex(
        index=reduction_methods, columns=multi_level_cols,
    )
    display(df_pivot)

dataset          SS-medium                                   \
metric                 FRR       RFR       RFP elapsed_time   
reduction_method                                              
None              0.000000  1.000000  0.023257       0.2928   
NSigma            0.192666  0.975926  0.028290       0.6013   
Birch             0.185164  0.974074  0.027839       0.3887   
K-S test          0.128849  0.966667  0.025904       0.6210   
FluxInfer-AD      0.128849  0.966667  0.025904       0.6230   
HDBSCAN-SBD       0.685723  0.254198  0.018476       0.7164   
HDBSCAN-PEARSON   0.691715  0.249259  0.018529       0.9246   
TSifter           0.462100  0.858642  0.037965       0.9544   

dataset                                SS-large                      \
metric           num_metrics(reduced)       FRR       RFR       RFP   
reduction_method                                                      
None                              184  0.000000  1.000000  0.005832   
NSigma                            148  0.333519  0.954857  0.008298   
Birch                             150  0.250693  0.950669  0.007350   
K-S test                          160  0.119126  0.966663  0.006418   
FluxInfer-AD                      160  0.119126  0.966663  0.006418   
HDBSCAN-SBD                        57  0.842580  0.184034  0.006400   
HDBSCAN-PEARSON                    56  0.835740  0.185404  0.006221   
TSifter                            99  0.554422  0.803921  0.010410   

dataset                                             
metric           elapsed_time num_metrics(reduced)  
reduction_method                                    
None                   0.4138                 1352  
NSigma                 0.9750                  902  
Birch                  0.7800                 1013  
K-S test               1.0577                 1192  
FluxInfer-AD           1.0598                 1192  
HDBSCAN-SBD            4.8919                  212  
HDBSCAN-PEARSON       62.7099                  222  
TSifter                2.7958                  605

In [41]:
tsdr_s = df_pivot.style
tsdr_s.clear()
tsdr_s.format(precision=3)
print(tsdr_s.to_latex(
))

\begin{tabular}{lrrrrrrrrrr}
dataset & \multicolumn{5}{r}{SS-medium} & \multicolumn{5}{r}{SS-large} \\
metric & FRR & RFR & RFP & elapsed_time & num_metrics(reduced) & FRR & RFR & RFP & elapsed_time & num_metrics(reduced) \\
reduction_method &  &  &  &  &  &  &  &  &  &  \\
None & 0.000 & 1.000 & 0.023 & 0.293 & 184 & 0.000 & 1.000 & 0.006 & 0.414 & 1352 \\
NSigma & 0.193 & 0.976 & 0.028 & 0.601 & 148 & 0.334 & 0.955 & 0.008 & 0.975 & 902 \\
Birch & 0.185 & 0.974 & 0.028 & 0.389 & 150 & 0.251 & 0.951 & 0.007 & 0.780 & 1013 \\
K-S test & 0.129 & 0.967 & 0.026 & 0.621 & 160 & 0.119 & 0.967 & 0.006 & 1.058 & 1192 \\
FluxInfer-AD & 0.129 & 0.967 & 0.026 & 0.623 & 160 & 0.119 & 0.967 & 0.006 & 1.060 & 1192 \\
HDBSCAN-SBD & 0.686 & 0.254 & 0.018 & 0.716 & 57 & 0.843 & 0.184 & 0.006 & 4.892 & 212 \\
HDBSCAN-PEARSON & 0.692 & 0.249 & 0.019 & 0.925 & 56 & 0.836 & 0.185 & 0.006 & 62.710 & 222 \\
TSifter & 0.462 & 0.859 & 0.038 & 0.954 & 99 & 0.554 & 0.804 & 0.010 & 2.796 & 605 \\
\end{tabular}



## Join tsdr and localization

In [32]:
joined_df = loc_tsdr_df.set_index(["target_app", "middleware", "localization_method"]).join(tsdr_df.set_index(["target_app", "middleware", "reduction_method"]), how="inner", on=['target_app', 'middleware', 'reduction_method'], lsuffix='_loc', rsuffix='_tsdr')
joined_df["elapsed_time_sum"] = joined_df["elapsed_time_loc"] + joined_df["elapsed_time_tsdr"]
joined_df.reset_index().set_index(['target_app', 'middleware', 'localization_method', 'reduction_method']).sort_index()

KeyError: "None of ['target_app', 'middleware'] are in the columns"

### Add random selection

In [45]:
tsdr_df

,dataset,reduction_method,FRR,RFR,RFP,elapsed_time,num_metrics(reduced),num_metrics(filtered),num_metrics(raw),num_ground_truth_metrics
0,SS-medium,None,0.000000,1.000000,0.023257,0.2928,184,184,499,8.400000
1,SS-large,None,0.000000,1.000000,0.005832,0.4138,1352,1352,6839,13.014815
2,SS-medium,TSifter,0.462100,0.858642,0.037965,0.9544,99,184,499,8.400000
3,SS-large,TSifter,0.554422,0.803921,0.010410,2.7958,605,1352,6839,13.014815
4,SS-medium,NSigma,0.192666,0.975926,0.028290,0.6013,148,184,499,8.400000
5,SS-large,NSigma,0.333519,0.954857,0.008298,0.9750,902,1352,6839,13.014815
6,SS-medium,Birch,0.185164,0.974074,0.027839,0.3887,150,184,499,8.400000
7,SS-large,Birch,0.250693,0.950669,0.007350,0.7800,1013,1352,6839,13.014815
8,SS-medium,K-S test,0.128849,0.966667,0.025904,0.6210,160,184,499,8.400000
9,SS-large,K-S test,0.119126,0.966663,0.006418,1.0577,1192,1352,6839,13.014815


In [47]:
from math import comb

def ac_k_random_selection(num_metrics: int, k: int, num_found_ground_truth: float) -> float:
    n, g = num_metrics, num_found_ground_truth
    prob_single_correct = g / n
    prob_at_least_one_correct = 1 - (1 - prob_single_correct) ** k
    return prob_at_least_one_correct

tsdr_df.apply(lambda x: ac_k_random_selection(x['num_metrics(reduced)'], 5, x['num_ground_truth_metrics']), axis=1)

0     0.208350
1     0.047214
2     0.358103
3     0.103031
4     0.253348
5     0.070092
6     0.250348
7     0.062609
8     0.236347
9     0.053413
10    0.236347
11    0.053413
12    0.549384
13    0.271509
14    0.556295
15    0.260714
dtype: float64

In [ ]:
import seaborn as sns

def wrap(label: str):
    match label.get_text():
        case "FluxInfer-AD":
            return "FluxInfer-\nAD"
        case "HDBSCAN-SBD":
            return "HDBSCAN-\nSBD"
        case "HDBSCAN-PEARSON":
            return "HDBSCAN-\nPEARSON"
    return label

fig, ax = plt.subplots(1, 1, figsize=(8, 5))
sns.pointplot(data=joined_df[loc_tsdr_df["middleware"] == False], x="reduction_method", y="ac@5", hue="localization_method", markers=["o", "s"], linestyles=["", ""], order=["None", "NSigma", "Birch", "K-S test", "FluxInfer-AD", "HDBSCAN-SBD", "HDBSCAN-PEARSON", "TSifter"], ax=ax)

ax.set_xlabel("Reduction Method", fontsize=12)
ax.set_ylabel("AC@5", fontsize=12)
ax.set_xticklabels([wrap(l) for l in ax.get_xticklabels()], rotation=0, fontsize=10)
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=10)
ax.legend(title="Localization Method")
sns.move_legend(
    ax, "lower center",
    bbox_to_anchor=(.5, 1), ncol=2, title="Localization Method", frameon=False, fontsize=10, title_fontsize=12
)

: 

In [53]:
num_metrics = pd.Series([100, 90, 80])
num_found_metrics = pd.Series([2, 3, 1])

def get_scores_of_random_selection(num_metrics: pd.Series, num_found_metrics: pd.Series, max_k: int = 5):

    def ac_k(num_metrics, num_found_ground_truth, k: int):
        n, g = num_metrics, num_found_ground_truth
        prob_single_correct = g / n
        prob_at_least_one_correct = 1 - (1 - prob_single_correct) ** k
        return prob_at_least_one_correct.mean()

    def avg_k(ac_k_) -> dict:
        return {k: sum([ac_k_[j] for j in range(1, k + 1)]) / k for k in range(1, max_k + 1)}

    ac_k_ = {k: ac_k(num_metrics, num_found_metrics, k) for k in range(1, max_k + 1)}
    avg_k_ = avg_k(ac_k_)

    return {f"AC_{k}": v for k, v in ac_k_.items()}, {f"AVG_{k}": v for k, v in avg_k_.items()}

get_scores_of_random_selection(num_metrics, num_found_metrics)

({'AC_1': 0.021944444444444433,
  'AC_2': 0.04333310185185187,
  'AC_3': 0.06418163560956791,
  'AC_4': 0.08450523611095033,
  'AC_5': 0.10431863564118209},
 {'AVG_1': 0.021944444444444433,
  'AVG_2': 0.032638773148148156,
  'AVG_3': 0.04315306063528807,
  'AVG_4': 0.053491104504203635,
  'AVG_5': 0.06365661073159933})